In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai


In [ ]:
import os
GEMINI_API_KEY = os.getenv('GEMINI')
genai.configure(api_key=GEMINI_API_KEY)

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3,google_api_key=GEMINI_API_KEY)

In [5]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
    llm = llm
)

In [6]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    llm = llm
)

In [7]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

In [8]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [9]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [10]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [11]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=2,
    memory=True,
    max_rpm=15,
    embedder={
            "provider": "google",
            "config":{
                "model": 'models/text-embedding-004',
                "task_type": "retrieval_document",
                "title": "Embeddings for Embedchain"
            }
    }    
)

In [12]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Sulaiman",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Sulaiman from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"} 

Assembling and Activating Your CrewAI Team - crewAI
 Skip to content
 crewAI
 Assembling and Activating Your CrewAI Team
 Initializing search
 crewAI
 crewAI
 crewAI
 Home
 Core Concepts
 Core Concepts
 Agents
 Tasks
 Tools
 Processes
 Crews
 Collaboration
 Training
 Memory
 Using LangChain Tools
 Using LlamaIndex Tools
 How to Guides
 How to

In [13]:
from IPython.display import Markdown
Markdown(result)

Hi Sulaiman,

Thank you for reaching out to us. I'd be happy to assist you with setting up a Crew and adding memory to it.

To add memory to your Crew, you can use the `memory` parameter when creating your Agent. Here's an example:

```python
from crewai import Agent

# Create an agent with memory enabled
agent = Agent(memory=True)
```

When you enable memory for an agent, it will remember the context of previous interactions and use that information to inform its responses in future interactions. This can be useful for tasks that require the agent to maintain a consistent state, such as customer service or dialogue generation.

Here are some additional resources that you may find helpful:

* [Creating a Crew and kicking it off](https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/)
* [Agent class documentation](https://crewai.com/docs/api/agent/)

Please let me know if you have any other questions. I'm here to help!

Best regards,

[Your name]